In [61]:
import pandas as pd
import chardet
import pycountry
import numpy as np
from datetime import datetime
from geopy.geocoders import Nominatim


In [62]:
current_date = datetime.now().strftime("%Y%m%d")
current_date

'20241122'

## Investigating countries not recognized by Tableau 
Put the source data in tableau and made two csvs of countries it couldn't recognize/map. Then make a list of unique countries between recipients and suppliers. This will give us a full list of the countries in our data that Tableau doesn't recognize so we can decide how to handle them. 

In [63]:
with open("int_data/unidentified_suppliers.csv", "rb") as f:
    supp_encoding = chardet.detect(f.read())
    print(supp_encoding["encoding"])

UTF-16


In [64]:
unid_supp_df = pd.read_csv('int_data/unidentified_suppliers.csv', encoding = supp_encoding['encoding'], sep='\t').reset_index()
unid_rec_df = pd.read_csv('int_data/unidentified_recipients.csv', encoding = supp_encoding['encoding'],sep='\t').reset_index()

unid_supp_df.head()

,index,Supplier,Number ordered
0,0,Yugoslavia,2198.0
1,1,unknown supplier(s),5390.0
2,2,United Nations**,4.0
3,3,Soviet Union,845420.0
4,4,South Yemen,12.0


In [65]:
# all countries in our source data not recognized by tableau
unique_unidentified_countries = list(set(unid_supp_df['Supplier']).union(set(unid_rec_df['Recipient'])))
unique_unidentified_countries

[nan,
 'unknown recipient(s)',
 '3',
 'ELF (Ethiopia)*',
 'unknown supplier(s)',
 'NLA (Macedonia)*',
 'Anti-Castro rebels (Cuba)*',
 'Contras (Nicaragua)*',
 'Amal (Lebanon)*',
 '0.25',
 'Soviet Union',
 'Indonesia rebels*',
 'Viet Minh (France)*',
 'SPLA (Sudan)*',
 'Armas (Guatemala)*',
 'Katanga',
 'Darfur rebels (Sudan)*',
 'RPF (Rwanda)*',
 'Hezbollah (Lebanon)*',
 'Libya GNC',
 'Hamas (Palestine)*',
 'Yugoslavia',
 'FMLN (El Salvador)*',
 'Viet Cong (South Vietnam)*',
 'UIC (Somalia)*',
 'FRELIMO (Portugal)*',
 'Libya HoR*',
 'PKK (Turkiye)*',
 'South Vietnam',
 'FNLA (Angola)*',
 'Pathet Lao (Laos)*',
 'Houthi rebels (Yemen)*',
 'PAIGC (Portugal)*',
 'NTC (Libya)*',
 'African Union**',
 'Yemen Arab Republic (North Yemen)',
 'Ukraine Rebels*',
 'European Union**',
 'Czechoslovakia',
 'MPLA (Portugal)*',
 'PLO (Israel)*',
 'South Yemen',
 'Syria rebels*',
 'MTA (Myanmar)*',
 'GUNT (Chad)*',
 'Mujahedin (Afghanistan)*',
 'ANC (South Africa)*',
 'United Wa State (Myanmar)*',
 'LRA 

### Read in our source data about arms trade
We will read in our data, and then compare the countries in the arms trade data to the countries recognized by Tableau.

In [66]:
with open("raw_data/trade-register.csv", "rb") as f:
    result = chardet.detect(f.read())
    print(result["encoding"])

FileNotFoundError: [Errno 2] No such file or directory: 'raw_data/trade-register.csv'

In [ ]:
df = pd.read_csv('raw_data/trade-register.csv', encoding=result["encoding"])


In [ ]:
unique_values_source_data = pd.unique(df[['Supplier', 'Recipient']].values.ravel())

# Convert to list if needed
unique_values_source_data = unique_values_source_data.tolist()
unique_values_source_data

In [ ]:
tableau_identified_countries = [item for item in unique_values_source_data if item not in unique_unidentified_countries]
tableau_identified_countries

In [ ]:
# countries that don't have tableau matches
no_tableau_matches = df[df[['Supplier','Recipient']].isin(unique_unidentified_countries).any(axis=1)]
no_tableau_matches.sort_values('Year of order', ascending=False)

Because there have been countries created/changed since roughly the early 2000s we won't be looking at data before that time range. Here we will filter our data to orders placed after 1995 that have either the supplier or recipient not recognized by Tableau. We will then compare this to the list of countries previously created that are not recognized by Tableau to get the final list of countries in our data set that are in our range of time and not recognized.  

In [ ]:
# Filter rows where 'Year of order' > 1995
filtered_df = no_tableau_matches[no_tableau_matches['Year of order'] > 2000]

# Get distinct values from both 'Recipient' and 'Supplier' columns
distinct_values = pd.concat([filtered_df['Recipient'], filtered_df['Supplier']]).dropna().unique()

# Convert to a list
distinct_list = list(distinct_values)

print(distinct_list)

In [ ]:
set(unique_unidentified_countries).intersection(distinct_list)

## Categorize weapon descriptions
Make a high level category and a sub-category for each weapon description 

In [ ]:
category_mapping = {
    'Aircraft': [
        'fighter aircraft', 'bomber aircraft', 'transport aircraft', 'reconnaissance aircraft', 
        'trainer aircraft', 'AEW&C aircraft', 'SIGINT aircraft', 'trainer/combat aircraft', 
        'AGS aircraft', 'FGA aircraft', 'FGA/EW aircraft', 'airship', 'target tow aircraft', 'light aircraft', 'ground attack aircraft',
        'ASW aircraft', 'AEW aircraft', 'AEW/AGS aircraft', 'reconnaissance/SIGINT aircraft', 'transport ac/helicopter', 'transport/trainer aircraft', 'airship', 
        'light transport aircraft', 'heavy transport aircraft', 'tanker/transport aircraft', 'trainer/light aircraft', 'light/trainer aircraft'
    ],
    'Helicopters': [
        'combat helicopter', 'transport helicopter', 'ASW helicopter', 'light helicopter', 
        'AEW helicopter', 'anti-ship helicopter', 'SIGINT helicopter', 'helicopter', 'ASW Helicopter'
    ],
    'Unmanned Vehicles': [
        'UAV', 'armed UAV', 'reconnaissance AV', 'maritime patrol UAV', 'light aircraft/UAV', 'loitering munition' 
    ],
    'Naval Vessels': [
        'frigate', 'destroyer', 'corvette', 'patrol craft', 'submarine', 'cargo ship', 'tanker',
        'support ship', 'training ship', 'MCM ship', 'replenishment ship', 'minehunter', 'icebreaker', 
        'OPV', 'OPV/training ship', 'OPV/support ship', 'gunboat', 'landing ship', 'salvage ship', 
        'survey ship', 'cruiser', 'aircraft carrier', 'minelayer', 'replenishment tanker', 
        'corvette/minesweeper', 'OPV/transport ship', 'support/landing ship', 'landing/patrol craft',
        'transport ship', 'FAC (Fast Attack Craft)', 'minesweeper', 'landing craft', 'tug', 'nuclear submarine', 'OPV/tug',
        'frigate/landing ship', 'SSB (Ballistic Missile Submarine)', 'SIGINT ship', 'patrol craft/transport craft', 'transport craft',
        'support craft', 'cargo craft', 'midget submarine', 'icebreaker/OPV', 'FAC', 'SSB'
    ],
    'Ground Vehicles/Artillery': [
        'tank', 'light tank', 'armoured car', 'APC', 'IFV', 'AFSV', 'APV', 'AMV', 'ARV', 'AEV/ARV', 
        'armoured bridgelayer', 'training tank', 'tank turret', 'self-propelled gun chassis', 'tank chassis',
        'APC/APV', 'APC/IFV', 'APC/CP', 'IFV/AFSV', 'IFV/APC turret', 'APC turret', 'IFV turret', 'self-propelled AD system',
        'self-propelled gun', 'self-propelled MRL', 'towed gun', 'self-propelled AA gun', 'coastal defence system', 'AA gun (Anti-Aircraft Gun)',
        'towed MRL', 'ALV (Armored Logistics Vehicle)', 'naval gun', 'tank destroyer', 'mortar', 'self-propelled mortar', 'anti-aircraft gun',
        'SPG turret', 'ASW mortar', 'AEV (Armored Engineering Vehicle)', 'helicopter turret', 'anti-tank AV turret', 'self-propelled AA gun turret',
        'apc (Armored Personnel Carrier)', 'AA gun/SAM system', 'AA gun system', 'coastal defence gun', 'mortar turret', 'AA gun', 'ALV', 'AEV', 'apc'
    ],
    'Missiles/Rockets/Bombs': [
        'SAM', 'portable SAM', 'mobile SAM system', 'naval SAM system', 'SAM system', 'SAM/ABM', 'ABM missile',
        'SSM', 'SSM launcher', 'SSM TEL', 'anti-ship missile', 'anti-radar missile', 'anti-tank missile', 
        'guided bomb', 'guided rocket', 'guided rocket/SSM', 'guided rocket/ASM', 'SSM/ASM', 'anti-ship missile/SSM',
        'anti-ship/land-attack missile', 'land-attack missile', 'anti-tank missile/ASM', 'guided glide bomb',
        'anti-ship/ASW torpedo', 'anti-ship torpedo', 'SSM/anti-ship missile', 'ASW torpedo', 'ASW MRL', 
        'anti-ship missile/ASM', 'BVRAAM', 'BVRAAM/SAM', 'SRAAM', 'ASM', 'ABM/SAM system', 'naval MRL', 'ASM (Anti-Ship Missile)',
        'guided shell', 'ASW missile', 'anti-ship/anti-radar missile', 'naval mine/torpedo', 'ABM system', 'ASW rocket launcher',
        'anti-ship missile/ASM/SAM', 'SLBM (Submarine-Launched Ballistic Missile)', 'ASM', 'SLBM', 'anti-radar missile/ASM'
    ],
    'Radar/Detection Systems': [
        'air search radar', 'AGS radar', 'sea search radar', 'multi-function radar', 'air search system', 
        'fire control radar', 'air/sea search radar', 'artillery locating radar', 'ground surv radar', 
        'ground/sea search radar', 'AEW radar', 'MP aircraft radar', 'aircraft EO system', 
        'aircraft EO/radar system', 'air/ground surv radar', 'SAM system radar', 'multi-role radar', 'radar',
        'combat aircraft radar', 'combat heli radar', 'SONAR', 'submarine sonar', 'ASW sonar', 'SONAR system',
        'aircraft recce system', 'height-finding radar', 'ASW sonar', 'SONAR', 'recce satellite', 'EO search/fire control', 'EO system',
        'AGS/MP aircraft radar', 'Naval EO system', 'AGS/SIGINT system', 'surveillance satellite'
    ],
    'Specialized Equipment': [
        'aircraft engine', 'turbofan', 'turbojet', 'gas turbine', 'vehicle engine', 'AIP engine', 
        'air refuel system', 'maritime patrol aircraft', 'SIGINT system', 
        'SIGINT aircraft', 'AALS', 'VEHICLE ENGINE', 'turbojet', 'nuclear reactor', 'ship engine', 'turboprop'
    ],
    'Other': [
        'SSB' , 'training equipment'
    ]
}

# Expanded sub-category mappings
sub_category_mapping = {
    # Aircraft Sub-categories
    'Aircraft': {
        'Fighter/Combat Aircraft': ['fighter aircraft', 'trainer/combat aircraft', 'FGA aircraft', 'FGA/EW aircraft', 'ground attack aircraft', 'ASW aircraft'],
        'Bomber/Heavy Aircraft': ['bomber aircraft', 'heavy transport aircraft'],
        'Reconnaissance/Surveillance Aircraft': ['reconnaissance aircraft', 'AEW&C aircraft', 'SIGINT aircraft', 'AGS aircraft', 'light aircraft', 'AEW aircraft',
                                                'AEW/AGS aircraft', 'reconnaissance/SIGINT aircraft', 'airship'],
        'Trainer Aircraft': ['trainer aircraft', 'trainer/light aircraft', 'light/trainer aircraft', 'target tow aircraft'],
        'Transport Aircraft': ['transport aircraft', 'light transport aircraft', 'tanker/transport aircraft', 'transport ac/helicopter', 'transport/trainer aircraft']
    },
    # Helicopter Sub-categories
    'Helicopters': {
        'Combat Helicopters': ['combat helicopter', 'ASW helicopter', 'anti-ship helicopter', 'SIGINT helicopter'],
        'Transport/Utility Helicopters': ['transport helicopter', 'light helicopter'],
        'Specialized Helicopters': ['AEW helicopter', 'ASW Helicopter', 'helicopter']
    },
    # Unmanned Vehicles Sub-categories
    'Unmanned Vehicles': {
        'UAV/Drone': ['UAV', 'armed UAV', 'light aircraft/UAV', 'loitering munition'],
        'Reconnaissance/Surveillance UAV': ['reconnaissance AV', 'maritime patrol UAV']
    },
    'Naval Vessels' :{
        'Combat Vessel': ['frigate', 'destroyer', 'corvette', 'submarine', 'gunboat', 'cruiser', 'aircraft carrier', 
                         'corvette/minesweeper', 'FAC (Fast Attack Craft)', 'minesweeper', 'nuclear submarine', 'SSB (Ballistic Missile Submarine)',
                         'midget submarine', 'FAC', 'SSB'],
        'Support Vessel' : [ 'patrol craft', 'cargo ship', 'tanker',
        'support ship', 'training ship', 'MCM ship', 'replenishment ship', 'minehunter', 'icebreaker', 
        'OPV', 'OPV/training ship', 'OPV/support ship', 'landing ship', 'salvage ship', 
        'survey ship', 'minelayer', 'replenishment tanker', 'OPV/transport ship', 'support/landing ship', 
        'landing/patrol craft', 'transport ship', 'landing craft', 'tug', 'OPV/tug', 'frigate/landing ship', 'SIGINT ship', 'patrol craft/transport craft',
        'transport craft', 'support craft', 'cargo craft', 'icebreaker/OPV']
    },
    'Ground Vehicles/Artillery' : {
    'Main Battle Tanks': ['tank', 'light tank', 'training tank', 'tank chassis', 'tank destroyer'],
    'Armored Personnel Carriers (APC)': ['APC', 'APC/APV', 'APC/IFV', 'APC/CP', 'apc (Armored Personnel Carrier)', 'apc'],
    'Infantry Fighting Vehicles (IFV)': ['IFV', 'IFV/AFSV', 'IFV/APC turret', 'IFV turret'],
    'Armored Reconnaissance Vehicles (ARV)': ['ARV', 'AEV/ARV'],
    'Armored Fighting Support Vehicles (AFSV)': ['AFSV', 'AMV', 'ALV (Armored Logistics Vehicle)', 'ALV'],
    'Light Armored Vehicles (APV)': ['APV', 'armoured car'],
    'Self-Propelled Artillery': ['self-propelled gun chassis', 'self-propelled AD system', 'self-propelled gun', 'self-propelled MRL', 'self-propelled AA gun',
                                'AA gun (Anti-Aircraft Gun)', 'self-propelled mortar', 'SPG turret', 'self-propelled AA gun turret'],
    'Engineering Vehicles': ['armoured bridgelayer', 'AEV (Armored Engineering Vehicle)', 'AEV'],
    'Tank Components': ['tank turret'],
    'Artillery' : ['towed gun', 'mortar', 'coastal defence system', 'towed MRL', 'naval gun', 'anti-aircraft gun', 'ASW mortar', 'helicopter turret', 'anti-tank AV turret',
                  'APC turret', 'AA gun/SAM system', 'AA gun system', 'coastal defence gun', 'mortar turret', 'AA gun']
    },
    'Missiles/Rockets/Bombs' : {
    'Surface-to-Air Missiles (SAM)': [
        'SAM', 'portable SAM', 'mobile SAM system', 'naval SAM system', 'SAM system', 'SAM/ABM', 'ABM missile', 'ABM/SAM system', 'ASM (Anti-Ship Missile)', 'ABM system'
    ],
    'Surface-to-Surface Missiles (SSM)': [
        'SSM', 'SSM launcher', 'SSM TEL', 'SSM/ASM', 'SSM/anti-ship missile', 'anti-ship missile/SSM', 'land-attack missile'
    ],
    'Air-to-Air Missiles (AAM)': [
        'BVRAAM', 'SRAAM', 'BVRAAM/SAM'
    ],
    'Anti-Ship Missiles (ASM)': [
        'anti-ship missile', 'anti-ship missile/ASM', 'anti-ship/ASW torpedo', 'anti-ship torpedo', 'SSM/anti-ship missile', 'anti-ship/anti-radar missile',
        'naval mine/torpedo', 'anti-ship missile/ASM/SAM', 'ASM'
    ],
    'Anti-Radar Missiles': [
        'anti-radar missile', 'anti-radar missile/ASM'
    ],
    'Anti-Tank Missiles': [
        'anti-tank missile', 'anti-tank missile/ASM'
    ],
    'Guided Bombs and Rockets': [
        'guided bomb', 'guided rocket', 'guided glide bomb', 'guided rocket/SSM', 'guided rocket/ASM', 'guided shell'
    ],
    'Land-Attack Missiles': [
        'anti-ship/land-attack missile', 'land-attack missile'
    ],
    'Anti-Submarine Warfare (ASW) Weapons': [
        'ASW torpedo', 'ASW MRL', 'anti-ship/ASW torpedo', 'ASW missile', 'ASW rocket launcher'
    ],
    'Naval Rocket Launchers': [
        'naval MRL'
    ],
    'Ballistic Missiles' : ['SLBM (Submarine-Launched Ballistic Missile)', 'SLBM']    
    
    },
    'Radar/Detection Systems' : {
    'Air Search Radar': [
        'air search radar', 'AEW radar', 'air search system', 'air/sea search radar', 'air/ground surv radar'
    ],
    'Sea Search Detection Systems': [
        'sea search radar', 'ground/sea search radar', 'ASW sonar', 'SONAR', 'submarine sonar', 'AGS/MP aircraft radar'
    ],
    'Ground Surveillance Radar': [
        'ground surv radar', 'ground/sea search radar', 'air/ground surv radar', 'AGS/SIGINT system'
    ],
    'Multi-function Radar': [
        'multi-function radar', 'multi-role radar'
    ],
    'Fire Control Radar': [
        'fire control radar', 'SAM system radar', 'AGS radar', 'EO system', 'EO search/fire control', 'Naval EO system'
    ],
    'Artillery/Weapon Locating Radar': [
        'artillery locating radar'
    ],
    'Aircraft-Mounted Systems': [
        'MP aircraft radar', 'aircraft EO system', 'aircraft EO/radar system', 'combat aircraft radar', 'combat heli radar',
        'aircraft recce system'
    ],
    'Other Detection Systems': ['height-finding radar', 'recce satellite', 'surveillance satellite'
                               ]
    },
    'Specialized Equipment' : {
    'Aircraft Engines': [
        'aircraft engine', 'turbofan', 'turbojet', 'turboprop'
    ],
    'Vehicle Engines': [
        'vehicle engine', 'VEHICLE ENGINE', 'gas turbine', 'AIP engine', 'nuclear reactor', 'ship engine'
    ],
    'Refueling and Logistics Systems': [
        'air refuel system', 'AALS'
    ],
    'Intelligence and Surveillance Systems': [
        'SIGINT system', 'SIGINT aircraft', 'maritime patrol aircraft'
    ]
}
    # Add more sub-categories as needed in similar structure...
}

# The mapping functions will work the same, now with the expanded mappings.

def map_category(description):
    for category, items in category_mapping.items():
        if description in items:
            return category
    return 'Unknown'

def map_sub_category(description):
    for category, sub_categories in sub_category_mapping.items():
        for sub_category, items in sub_categories.items():
            if description in items:
                return sub_category
    return 'Unknown'

# Applying the mapping functions
df['Category'] = df['Weapon description'].apply(map_category)
df['Sub-Category'] = df['Weapon description'].apply(map_sub_category)



In [ ]:
# Creating a DataFrame with three columns: 'Description', 'Category', and 'Sub-category' for exporting to csv for sharing 
csv_mapping_data = {'Category': [], 'Sub-category': [], 'Description': []}

for category, subcategories in sub_category_mapping.items():
    for sub_category, descriptions in subcategories.items():
        for description in descriptions:
            csv_mapping_data['Category'].append(category)
            csv_mapping_data['Sub-category'].append(sub_category)
            csv_mapping_data['Description'].append(description)

# Generating the DataFrame
df_three_columns = pd.DataFrame(csv_mapping_data)
df_three_columns.head()

In [ ]:
# Save this mapping to a csv
df_three_columns.to_csv('final_data/description_mapping.csv')

In [ ]:
# drop nonsense columns
df.drop([' .1', ' .2', ' '], axis=1, inplace=True)
df.head()

# Add WDI indicators
We will add world bank country indicators do our database related to military expenditures and also bring in the region and income group of each country.

We'll start by reading in the indicator data (WDICSV) and the country info (WDICountry).

From the WDICountry table we'll keep the table name, region, and income group for joining to our data later 

In [ ]:
wdi_df = pd.read_csv('raw_data/WDICSV.csv') 
country_df = pd.read_csv('raw_data/WDICountry.csv')
country_df.head()

In [ ]:
country_region_mapping = country_df[['Table Name', 'Region', 'Income Group', '2-alpha code']]
country_region_mapping = country_region_mapping.rename(columns={'2-alpha code': 'two_alpha_code'})

country_region_mapping

In [ ]:
country_region_mapping[country_region_mapping['two_alpha_code']=='TR']

In [ ]:
wdi_df = wdi_df.merge(country_df[['Table Name','Region','Income Group', 'Long Name']], how='left', left_on='Country Name', right_on = 'Table Name')
wdi_df = wdi_df.drop('Table Name', axis=1)

In [ ]:
wdi_df_long = pd.melt(wdi_df, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'Region', 'Income Group'],
                      var_name = 'Year',
                      value_name = 'indicator_value')
wdi_df_long.head()

In [ ]:
indicators = [
    "Military expenditure (% of GDP)",
    "Military expenditure (% of general government expenditure)",
    "Military expenditure (current LCU)",
    "Armed Forces Personnel (% of total labor force)",
    "Armed Forces personnel, total",
    "Arms exports (SIPRI trend indicator values)",
    "Arms imports (SIPRI trend indicator values)",
    "Military expenditure (Current USD)",
    "Armed forces personnel (% of total labor force)",
    "Armed forces personnel, total"

]

In [ ]:
def pivot_and_save_indicators(df, indicators, csv_name=None):
    # Filter the DataFrame to include only the specified indicators
    filtered_df = df[df['Indicator Name'].isin(indicators)]
    
    # Pivot the DataFrame
    pivoted_df = filtered_df.pivot(
        index=["Country Name", "Country Code", "Year", "Region", "Income Group"],
        columns="Indicator Name",
        values="indicator_value"
    ).reset_index()
    
    if csv_name != None:
    # Save the pivoted DataFrame to a CSV file
        pivoted_df.to_csv(csv_name, index=False)
    
    # Return the pivoted DataFrame
    return pivoted_df

In [ ]:
arms_indicators = pivot_and_save_indicators(wdi_df_long, indicators)

In [ ]:
non_region_df = arms_indicators[arms_indicators['Region'].notna()]
non_region_df.info()

In [ ]:
# Convert 'Column1' to numeric with errors coerced to NaN
non_region_df['Year'] = pd.to_numeric(non_region_df['Year'], errors='coerce')

# Drop rows where 'Column1' is NaN
non_region_df = non_region_df.dropna(subset=['Year'])

# Convert the column to integer type
non_region_df['Year'] = non_region_df['Year'].astype(int)

In [ ]:
non_region_df = non_region_df[non_region_df['Year']>=2000]

In [ ]:
# Filter rows where 'Region' is not NaN
filtered_df = non_region_df[non_region_df['Region'].notna()]

# Get unique values from the 'Country Name' column
unique_countries_wdi = filtered_df['Country Name'].unique()

# Convert to a list (optional)
unique_countries_list = unique_countries_wdi.tolist()

print(unique_countries_list)

In [ ]:
tableau_countries_not_in_indicators_data = [item for item in tableau_identified_countries if item not in unique_countries_list]
tableau_countries_not_in_indicators_data

In [ ]:
mapping_wbi_to_sipri = {
    'Russian Federation': 'Russia',
 'Slovak Republic': 'Slovakia',
 'Bosnia and Herzegovina': 'Bosnia-Herzegovina',
 'Turkey': 'Turkiye',
 'Türkiye':'Turkiye',
 'Egypt, Arab Rep.': 'Egypt',
 'United Arab Emirates': 'UAE',
 'Syrian Arab Republic': 'Syria',
 'Korea, Rep.': 'South Korea',
 'Bahamas, The': 'Bahamas',
 'Macao SAR, China': 'Taiwan',
 'Côte d’Ivoire': "Cote d'Ivoire",
 'Venezuela, RB': 'Venezuela',
 'Iran, Islamic Rep.': 'Iran',
 'Brunei Darussalam': 'Brunei',
 'Kyrgyz Republic': 'Kyrgyzstan',
 'Congo, Rep.': 'Congo',
 'Congo, Dem. Rep.': 'DR Congo',
 "Korea, Dem. People's Rep.": 'North Korea',
 'Eswatini': 'eSwatini',
 'Gambia, The': 'Gambia',
 'Lao PDR': 'Laos',
 'Micronesia, Fed. Sts.': 'Micronesia',
 'Cyprus': 'Northern Cyprus',
 'West Bank and Gaza': 'Palestine',
 'St. Kitts and Nevis': 'Saint Kitts and Nevis',
 'St. Vincent and the Grenadines': 'Saint Vincent',
 'Yemen, Rep.': 'Yemen'}


In [ ]:
non_region_df['sipri_country_mapping'] = non_region_df['Country Name'].apply(
    lambda x: x if x not in mapping_wbi_to_sipri else mapping_wbi_to_sipri[x]
)

In [ ]:
country_region_mapping['sipri_country_mapping'] = country_region_mapping['Table Name'].apply(
    lambda x: x if x not in mapping_wbi_to_sipri else mapping_wbi_to_sipri[x]
)
country_region_mapping.drop(columns=['Table Name'], inplace=True)
country_region_mapping

In [ ]:
non_region_df.head()

In [ ]:
#non_region_df.to_csv('final_data/arms_indicator_data.csv')

In [ ]:
df.head()

In [ ]:
df = df.drop(columns=['.1', '.2'], errors='ignore')

df['Year of order'] = df['Year of order'].astype(int)

In [ ]:
df.info()

In [ ]:
df.to_csv('final_data/cleaned_transfer_data.csv')

## Restructure dataframe
We need to have a primary country name so we restructure the dataframe so that there are two rows per transaction one where the supplier is the primary country and one where the recipient is the primary country. 

In [ ]:
# Melt the DataFrame to create rows for Supplier and Recipient
df["transaction_id"] = range(1, len(df) + 1)

supplier_df = df.copy()
supplier_df["Country"] = supplier_df["Supplier"]
supplier_df["Country Role"] = "Supplier"
#supplier_df.drop(columns=["Supplier", "Recipient"], inplace=True)

recipient_df = df.copy()
recipient_df["Country"] = recipient_df["Recipient"]
recipient_df["Country Role"] = "Recipient"
#recipient_df.drop(columns=["Supplier", "Recipient"], inplace=True)





In [ ]:
# Combine the two DataFrames
transformed_df = pd.concat([supplier_df, recipient_df], ignore_index=True)

# Reorder columns to have "Country" and "Country Role" as the first two columns
transformed_df["SIPRI TIV for total order - Pos_Neg"] = np.where(
    transformed_df["Country Role"] == "Recipient",
    transformed_df["SIPRI TIV for total order"] * -1.0,
    transformed_df["SIPRI TIV for total order"]
)

drop_cols = ['Number delivered','Year(s) of delivery','SIPRI TIV of delivered weapons']
transformed_df = transformed_df.drop(columns=drop_cols)
transformed_df["Partner Country"] = np.where(
    transformed_df["Recipient"] != transformed_df["Country"],  # Condition
    transformed_df["Recipient"],                  # Value if True
    transformed_df["Supplier"]                   # Value if False
)
transformed_df = transformed_df.drop(columns=["Supplier", "Recipient"])
cols = ["transaction_id", "Country", "Country Role", "Partner Country"] + [col for col in transformed_df.columns if col not in ["transaction_id", "Country", "Country Role", "Partner Country"]]

transformed_df = transformed_df[cols]

transformed_df.sort_values('transaction_id').head()


In [ ]:
sipri_wdi_merge = df.merge(non_region_df, how='left', left_on=['Recipient','Year of order'], right_on=['sipri_country_mapping','Year'])

In [ ]:
non_region_df['Year'] = non_region_df['Year'].astype(int)

non_region_df.info()

In [ ]:
melt_wdi_sipri_merge = transformed_df.merge(non_region_df, how='left', left_on=['Country','Year of order'], right_on=['sipri_country_mapping','Year'])
melt_wdi_sipri_merge.info()

In [ ]:
drop_cols = ['Year', 'Region', 'Income Group', 'Country Name', 'sipri_country_mapping']
melt_wdi_sipri_merge = melt_wdi_sipri_merge.drop(columns=drop_cols)
melt_wdi_sipri_merge= melt_wdi_sipri_merge.merge(country_region_mapping, how='left', left_on='Country', right_on='sipri_country_mapping')
melt_wdi_sipri_merge.drop(columns=['sipri_country_mapping'], inplace=True)
melt_wdi_sipri_merge.rename(mapper={'Region':'Primary Country Region',
                                    'Income Group':'Primary Country Region',
                                    'two_alpha_code':'primary_two_alpha_code',
                                    #'sipri_country_mapping_x':'sipri_country_mapping'
                                   }, axis=1, inplace=True)
melt_wdi_sipri_merge= melt_wdi_sipri_merge.merge(country_region_mapping, how='left', left_on='Partner Country', right_on='sipri_country_mapping')
melt_wdi_sipri_merge.drop(columns=['sipri_country_mapping'], inplace=True)
melt_wdi_sipri_merge.rename(mapper={'Region':'Partner Country Region',
                                    'Income Group':'Partner Country Region',
                                    'two_alpha_code':'partner_two_alpha_code',
                                   # 'sipri_country_mapping_x':'sipri_country_mapping'
                                   }, axis=1, inplace=True)

melt_wdi_sipri_merge

In [ ]:
melt_wdi_sipri_merge.info()

## Add Latitude and Longitude for each country
We need latitude and longitude to create lines between countries in tableau

In [ ]:
lat_long_df = pd.read_csv('raw_data/countries_lat_long.csv')
lat_long_df

In [ ]:
lat_long_df[lat_long_df['country']=='TR']

In [ ]:
lat_long_df = lat_long_df[~lat_long_df['country'].isna()]

In [ ]:
melt_wdi_sipri_merge= melt_wdi_sipri_merge.merge(lat_long_df, how='left', left_on='primary_two_alpha_code', right_on='country')
melt_wdi_sipri_merge.drop(columns=['country','name'], inplace=True)
melt_wdi_sipri_merge.rename(mapper={'latitude':'Primary Latitude',
                             'longitude':'Primary Longitude'},
                             axis=1, inplace=True)

melt_wdi_sipri_merge= melt_wdi_sipri_merge.merge(lat_long_df, how='left', left_on='partner_two_alpha_code', right_on='country')
melt_wdi_sipri_merge.drop(columns=['country','name'], inplace=True)
melt_wdi_sipri_merge.rename(mapper={'latitude':'Partner Latitude',
                             'longitude':'Partner Longitude'},
                             axis=1, inplace=True)
melt_wdi_sipri_merge.head()

In [ ]:
melt_wdi_sipri_merge = melt_wdi_sipri_merge[melt_wdi_sipri_merge['Year of order']>=2000]
melt_wdi_sipri_merge.head()

In [ ]:
melt_wdi_sipri_merge[melt_wdi_sipri_merge['Primary Latitude'].isna()]

In [ ]:
#melt_wdi_sipri_merge = melt_wdi_sipri_merge.drop(columns=['sipri_country_mapping'])

melt_wdi_sipri_merge.to_csv('final_data/'+current_date+'one_row_one_country_data.csv')

In [ ]:
melt_wdi_sipri_merge.sort_values('transaction_id').head()

In [ ]:
melt_wdi_sipri_merge.info()

In [ ]:
sipri_wdi_merge.head()

In [ ]:
sipri_wdi_merge.info()

In [ ]:
sipri_wdi_merge.to_csv('final_data/final_nov21_sipri_recipient_indicator_merge.csv')

In [ ]:
sipri_wdi_merge[(sipri_wdi_merge['sipri_country_mapping'].isna())&(sipri_wdi_merge['Year of order']>2000)]

In [ ]:
sipri_wdi_merge[(sipri_wdi_merge['sipri_country_mapping'].isna())&(sipri_wdi_merge['Year of order']>2000)]['Recipient'].value_counts()

In [ ]:
non_region_df[(non_region_df['Year']==2007)&(non_region_df['sipri_country_mapping']=='Turkey')]

## Create csv with one row per year delivered

In [ ]:
delivered_df = df.dropna(subset=['First Year'])

delivered_df['First Year'] = delivered_df['First Year'].astype(int)

# Expand rows based on `Num years Delivered In`
annual_delivery_df = delivered_df.loc[delivered_df.index.repeat(delivered_df['Num years Delivered In'])].copy()

# Calculate the `year_delivered` for each expanded row
annual_delivery_df['year_delivered'] = annual_delivery_df.groupby(level=0).cumcount() + annual_delivery_df['First Year']
annual_delivery_df['num_delivered_in_year'] = annual_delivery_df['Number delivered']/annual_delivery_df['Num years Delivered In']
annual_delivery_df['SIPRI_TIV_delivered_in_year'] = annual_delivery_df['num_delivered_in_year']*annual_delivery_df['SIPRI TIV per unit']


In [ ]:
annual_delivery_df = annual_delivery_df.rename(columns={'Number delivered':'Total number delivered'})
annual_delivery_df.head()

In [ ]:
annual_delivery_df.to_csv('final_data/annual_delivery_data.csv')